In [34]:
import pandas as pd # use for data manipulation and analysis
import numpy as np # use for multi-dimensional array and matrix

import seaborn as sns # use for high-level interface for drawing attractive and informative statistical graphics 
import matplotlib.pyplot as plt # It provides an object-oriented API for embedding plots into applications
%matplotlib inline 
# It sets the backend of matplotlib to the 'inline' backend:
import plotly.express as px
import time # calculate time 

from sklearn.linear_model import LogisticRegression # algo use to predict good or bad
from sklearn.naive_bayes import MultinomialNB # nlp algo use to predict good or bad

from sklearn.model_selection import train_test_split # spliting the data between feature and target
from sklearn.metrics import classification_report # gives whole report about metrics (e.g, recall,precision,f1_score,c_m)
from sklearn.metrics import confusion_matrix # gives info about actual and predict
from nltk.tokenize import RegexpTokenizer # regexp tokenizers use to split words from text  
from nltk.stem.snowball import SnowballStemmer # stemmes words
from sklearn.feature_extraction.text import CountVectorizer # create sparse matrix of words using regexptokenizes  
from sklearn.pipeline import make_pipeline # use for combining all prerocessors techniuqes and algos

from PIL import Image # getting images in notebook
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator# creates words colud

from bs4 import BeautifulSoup # use for scraping the data from website
from selenium import webdriver # use for automation chrome 
import networkx as nx # for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

import pickle# use to dump model 

import warnings # ignores pink warnings 
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [35]:
# Loading the dataset
fack_job = pd.read_csv('fake_job.csv')

In [36]:
fack_job.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [37]:
fack_job.tail()

,URL,Label
549341,23.227.196.215/,bad
549342,apple-checker.org/,bad
549343,apple-iclods.org/,bad
549344,apple-uptoday.org/,bad
549345,apple-search.info,bad


In [38]:
fack_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549346 entries, 0 to 549345
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549346 non-null  object
 1   Label   549346 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB


In [39]:
fack_job.isnull().sum() # there is no missing values

URL      0
Label    0
dtype: int64

In [40]:
#create a dataframe of classes counts
label_counts = pd.DataFrame(fack_job['Label'].value_counts())

In [41]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+')#to getting alpha only

In [42]:
fack_job.URL[0]

'nobell.it/70ffb52d079109dca5664cce6f317373782/login.SkyPe.com/en/cgi-bin/verification/login/70ffb52d079109dca5664cce6f317373/index.php?cmd=_profile-ach&outdated_page_tmpl=p/gen/failed-to-load&nav=0.5.1&login_access=1322408526'

In [43]:
# this will be pull letter which matches to expression
tokenizer.tokenize(fack_job.URL[0]) # using first row

['nobell',
 'it',
 'ffb',
 'd',
 'dca',
 'cce',
 'f',
 'login',
 'SkyPe',
 'com',
 'en',
 'cgi',
 'bin',
 'verification',
 'login',
 'ffb',
 'd',
 'dca',
 'cce',
 'f',
 'index',
 'php',
 'cmd',
 'profile',
 'ach',
 'outdated',
 'page',
 'tmpl',
 'p',
 'gen',
 'failed',
 'to',
 'load',
 'nav',
 'login',
 'access']

In [44]:
print('Getting words tokenized ...')
t0= time.perf_counter()
fack_job['text_tokenized'] = fack_job.URL.map(lambda t: tokenizer.tokenize(t)) # doing with all rows
t1 = time.perf_counter() - t0
print('Time taken',t1 ,'sec')

Getting words tokenized ...
Time taken 4.705801000003703 sec


In [45]:
fack_job.sample(5)

,URL,Label,text_tokenized
291747,biography.com/people/tex-avery-5540,good,"[biography, com, people, tex, avery]"
430449,shockya.com/news/2011/09/24/nyff-2011-movie-re...,good,"[shockya, com, news, nyff, movie, review, musi..."
164150,defaultface.livejournal.com/,good,"[defaultface, livejournal, com]"
219113,mylifeofcrime.wordpress.com/2006/03/01/this-da...,good,"[mylifeofcrime, wordpress, com, this, date, in..."
51015,www.lfy.ca/index.html,good,"[www, lfy, ca, index, html]"


In [46]:
stemmer = SnowballStemmer("english") # choose a language

In [ ]:
print('Getting words stemmed ...')
t0= time.perf_counter()
fack_job['text_stemmed'] = fack_job['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])
t1= time.perf_counter() - t0
print('Time taken',t1 ,'sec')


Getting words stemmed ...


In [ ]:
fack_job.sample(5)

In [ ]:
print('Getting joiningwords ...')
t0= time.perf_counter()
fack_job['text_sent'] = fack_job['text_stemmed'].map(lambda l: ' '.join(l))
t1= time.perf_counter() - t0
print('Time taken',t1 ,'sec')

In [ ]:
fack_job.sample(5)

### Visualization 
**1. Visualize some important keys using word cloud**

In [ ]:
#sliceing classes
bad_sites = fack_job[fack_job.Label == 'bad']
good_sites = fack_job[fack_job.Label == 'good']

mixed_sites = pd.concat([bad_sites[:100],good_sites[:100]])
mixed_sites

In [ ]:
bad_sites.head()

In [ ]:
good_sites.head()

* create a function to visualize the important keys from url 

In [ ]:
data = good_sites.text_sent
data.reset_index(drop=True, inplace=True)

In [ ]:
data = bad_sites.text_sent
data.reset_index(drop=True, inplace=True)

In [ ]:
#create cv object
cv = CountVectorizer()

In [ ]:
feature = cv.fit_transform(fack_job.text_sent) #transform all text which we tokenize and stemed

In [ ]:
feature[:5].toarray() # convert sparse matrix into array to print transformed features

In [ ]:
trainX, testX, trainY, testY = train_test_split(feature, fack_job.Label)

In [ ]:

# Sample data
texts = mixed_sites['text_sent']
labels =mixed_sites['Label']# 1 for positive, 0 for negative

le = LabelEncoder()
le.fit(['bad','good'])
labels = le.transform(labels)
# Tokenize and pad the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, padding='post')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Build a Bidirectional LSTM model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=32, input_length=padded_sequences.shape[1]))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=2, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')
loaded_model = pickle.load(open('fake.pkl', 'rb'))
# Make predictions on new text



In [ ]:
model.summary()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

In [ ]:
from sklearn import svm
clf = svm.SVC()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))


In [ ]:
X_train

In [ ]:
import pickle


def predict(url):
    result = loaded_model.predict([url])
    print(result)
predict('phlebolog.com.ua/libraries/joomla/results.php')
predict('https://www.python.org/downloads/release/python-390/')
